Naming Explanation :
XXXYYZZZ
XXX - ID number of a person who has done the signature. 
YY - Image sample number.
ZZZ - ID number of person whose signature is in photo.

Example: 
NFI-00602023 is an image of signature of person number 023 done by person 006. This is a forged signature.
NFI-02103021 is an image of signature of person number 021 done by person 021. This is a genuine signature.   

# Features Used ? :
# Centroid : Center of Image.  In case of a grayscale image you can use the pixels' gray values to calculate the weighted average position.
# Ratio ? the ratio of its width to its height
# Eccentricity ? a disparity on an image between the centre of the projected object and the projected location of the centre of an object
# Solidity ? Solidity is useful to quantify the amount and size of concavities in an object boundary. Holes are also often included
# Skewness : meausre of symmetric. if the skewness is negative, the histogram is negatively skewed. The positive skewness is the opposite.
# Kurtosis : is the average (or expected value) of the standardized data raised to the fourth power.

In [2]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import ndimage
from skimage.filters import threshold_otsu
import tensorflow as tf
from keras import models, layers, preprocessing
from sklearn.model_selection import train_test_split
import os

In [3]:
#use git repo to detect local directory
import git
repo = git.Repo('.', search_parent_directories=True)

In [4]:
#Preprocessing

def preProcessing(image_path):
    img = cv2.imread(image_path)
    resizedImg = cv2.resize(img,(512, 512))
    img_gray = cv2.cvtColor(resizedImg, cv2.COLOR_BGR2GRAY)
    # img_edge = cv2.Canny(img_gray, 50, 100)
    img_filtered = ndimage.gaussian_filter(img_gray, 0.5)
    image_norm = cv2.normalize(img_filtered, None, alpha=0,beta=255, norm_type=cv2.NORM_MINMAX)
    return image_norm

In [5]:
# img_edge = cv2.Canny(img_gray, 50, 100)
# (thresh, binImg) = cv2.threshold(img_filtered, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

In [6]:
# Output processed image into files

path_real = repo.working_tree_dir + "\\image\\REAL_001\\"
path_forge = repo.working_tree_dir + "\\image\\FORGE_001\\"

path_real_p = repo.working_tree_dir + "\\image_p\\REAL_001\\"
path_forge_p = repo.working_tree_dir + "\\image_p\\FORGE_001\\"

def preProc(path, path_p) :
    try:
        os_path = os.listdir(path)
        for img in os_path[:]:
            print("From : " + os.path.join(path, img))
            print("To : " + os.path.join(path_p, img))
            processed_img = preProcessing(os.path.join(path, img))
            status = plt.imsave(path_p + img, processed_img, cmap="gray")
            # status = cv2.imwrite(os.path.join(path_p, img), processed_img)
            print(status)
        return
    except Exception as e:
        print(f"Error : {e}")

In [7]:
preProc(path_real, path_real_p)
preProc(path_forge, path_forge_p)

From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL_001\00100001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL_001\00100001.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL_001\00101001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL_001\00101001.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL_001\00102001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL_001\00102001.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL_001\00103001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL_001\00103001.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL_001\00104001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL_001\00104001.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL_001\00105001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL_001\00105001.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-s

In [8]:
# Enumerate Label

label_name = ["REAL_001","FORGE_001"]

enumerated_label = {}

for i, class_label in enumerate(label_name) :
    enumerated_label[class_label] = i

enumerated_label

{'REAL_001': 0, 'FORGE_001': 1}

In [9]:
# LABELING

data = []
label = []
image_path = repo.working_tree_dir + "\\image_p\\"

def labeling(enumerated_label,image_path,data_img,label_img):
    for folder in os.listdir(image_path):
        label = enumerated_label[folder]

        for file in os.listdir(os.path.join(image_path, folder)):
            img_path = os.path.join(os.path.join(image_path, folder), file) 
            print(img_path)
            image = preProcessing(img_path)
            data_img.append(image)
            label_img.append(label)

In [10]:
labeling(enumerated_label,image_path,data,label)

e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\FORGE_001\02100001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\FORGE_001\02101001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\FORGE_001\02103001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\FORGE_001\02104001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\FORGE_001\02201001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\FORGE_001\02202001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\FORGE_001\02203001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\FORGE_001\02204001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\FORGE_001\02205001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL_001\00100001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL_001\00101001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL_001\00102001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL_001\00103001.png
e:\Kuliah\SMT 6\Citra\pcd-sign-recognition

In [11]:
print(data)

[array([[250, 250, 250, ..., 250, 250, 250],
       [250, 250, 250, ..., 250, 250, 250],
       [250, 250, 250, ..., 250, 250, 250],
       ...,
       [250, 250, 250, ..., 249, 249, 249],
       [250, 250, 250, ..., 249, 249, 249],
       [250, 250, 250, ..., 249, 249, 249]], dtype=uint8), array([[248, 248, 248, ..., 249, 249, 249],
       [248, 248, 248, ..., 249, 249, 249],
       [248, 248, 248, ..., 249, 249, 249],
       ...,
       [249, 249, 249, ..., 249, 249, 249],
       [249, 249, 249, ..., 249, 249, 249],
       [249, 249, 249, ..., 249, 249, 249]], dtype=uint8), array([[250, 250, 250, ..., 250, 250, 250],
       [250, 250, 250, ..., 250, 250, 250],
       [250, 250, 250, ..., 250, 250, 250],
       ...,
       [247, 247, 247, ..., 250, 250, 250],
       [247, 247, 247, ..., 250, 250, 250],
       [247, 247, 247, ..., 250, 250, 250]], dtype=uint8), array([[250, 250, 250, ..., 250, 250, 250],
       [250, 250, 250, ..., 250, 250, 250],
       [250, 250, 250, ..., 250, 250, 

In [12]:
data = np.array(data, dtype = 'float32')
label = np.array(label, dtype = 'int32')
# for reshaping

# Reshape to 1 channel (Grayscale) <- If needed
data = data.reshape(data.shape[0], data.shape[1], data.shape[2], 1)
data.shape

(15, 512, 512, 1)

In [13]:
print(data)

[[[[250.]
   [250.]
   [250.]
   ...
   [250.]
   [250.]
   [250.]]

  [[250.]
   [250.]
   [250.]
   ...
   [250.]
   [250.]
   [250.]]

  [[250.]
   [250.]
   [250.]
   ...
   [250.]
   [250.]
   [250.]]

  ...

  [[250.]
   [250.]
   [250.]
   ...
   [249.]
   [249.]
   [249.]]

  [[250.]
   [250.]
   [250.]
   ...
   [249.]
   [249.]
   [249.]]

  [[250.]
   [250.]
   [250.]
   ...
   [249.]
   [249.]
   [249.]]]


 [[[248.]
   [248.]
   [248.]
   ...
   [249.]
   [249.]
   [249.]]

  [[248.]
   [248.]
   [248.]
   ...
   [249.]
   [249.]
   [249.]]

  [[248.]
   [248.]
   [248.]
   ...
   [249.]
   [249.]
   [249.]]

  ...

  [[249.]
   [249.]
   [249.]
   ...
   [249.]
   [249.]
   [249.]]

  [[249.]
   [249.]
   [249.]
   ...
   [249.]
   [249.]
   [249.]]

  [[249.]
   [249.]
   [249.]
   ...
   [249.]
   [249.]
   [249.]]]


 [[[250.]
   [250.]
   [250.]
   ...
   [250.]
   [250.]
   [250.]]

  [[250.]
   [250.]
   [250.]
   ...
   [250.]
   [250.]
   [250.]]

  [[250.]
   [25

In [14]:
print(label)

[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]


In [15]:
# MODEL , still figuring out
model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation="relu",name='conv2d_1' ,
                        input_shape=(512,512,1))) #GrayScale :1 || RGB : 3
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_1"))
model.add(layers.Conv2D(64, (3,3), activation="relu",name='conv2d_2'))
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_2"))
model.add(layers.Conv2D(128, (3,3), activation="relu",name='conv2d_3'))
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_3"))
model.add(layers.Conv2D(256, (3,3), activation="relu",name='conv2d_4'))
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_4"))
model.add(layers.Conv2D(256, (3,3), activation="relu", name='conv2d_5'))
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_5"))
model.add(layers.Conv2D(512, (3,3), activation="relu",name='conv2d_6'))
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_6"))
model.add(layers.Flatten(name="flatten_1"))
model.add(layers.Dense(256, activation='relu', name="dense_1"))
model.add(layers.Dense(2, activation='relu', name="dense_2"))

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 510, 510, 32)      320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 255, 255, 32)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 253, 253, 64)      18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 126, 126, 64)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 124, 124, 128)     73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 62, 62, 128)      0

In [17]:
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.3)

In [18]:
res = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1/1 [==============================] - 6s 6s/step - loss: 6.4472 - accuracy: 0.6000 - val_loss: 6.4472 - val_accuracy: 0.6000
Epoch 2/5
1/1 [==============================] - 5s 5s/step - loss: 6.4472 - accuracy: 0.6000 - val_loss: 6.4472 - val_accuracy: 0.6000
Epoch 3/5
1/1 [==============================] - 4s 4s/step - loss: 6.4472 - accuracy: 0.6000 - val_loss: 6.4472 - val_accuracy: 0.6000
Epoch 4/5
1/1 [==============================] - 4s 4s/step - loss: 6.4472 - accuracy: 0.6000 - val_loss: 6.4472 - val_accuracy: 0.6000
Epoch 5/5
1/1 [==============================] - 6s 6s/step - loss: 6.4472 - accuracy: 0.6000 - val_loss: 6.4472 - val_accuracy: 0.6000


In [19]:
model.evaluate(x_test, y_test)

1/1 [==============================] - 1s 528ms/step - loss: 6.4472 - accuracy: 0.6000


[6.447237968444824, 0.6000000238418579]

# TODO : 
# - Find better model and datasets (not yet)
# - Better Preprocessing (still figuring out)
# - Consider multiple subjects or focus on 1 subject (agreed on 1 subject but couldnt find the suitable datasets)
# - Preprocessing tilt image (still figuring out if its possible or no)
# - Compare : Features // find edges (tried Canny but causing image data to be all zero and not detecting white part)